In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 1    # number of expression blocks
num_data = 5000   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
df.to_csv("result/rb_exact_50-1.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:00<00:00,  1.64it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     2.012429               0.0      0.607808
std      2.581801               0.0      0.256349
min      0.000020               0.0      0.124618
25%      0.228725               0.0      0.358065
50%      0.808438               0.0      0.681139
75%      2.995639               0.0      0.794783
max     10.617350               0.0      1.116472
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_rnd_50-1.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:27<00:00,  1.14it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     5.672553          0.008490      0.869363
std      4.052065          0.051887      0.154442
min      0.008170          0.000000      0.462962
25%      2.143000          0.000000      0.788311
50%      5.429408          0.000000      0.898069
75%      9.039523          0.000000      1.005212
max     15.706952          0.360569      1.232177
Number of infeasible solution: 3


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_heur_n1_50-1.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.24it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   104.881624               0.0      0.074986
std     69.592172               0.0      0.032932
min     11.253633               0.0      0.058766
25%     42.798825               0.0      0.061067
50%    106.690124               0.0      0.062082
75%    141.728357               0.0      0.070550
max    304.232321               0.0      0.183562
Number of infeasible solution: 0


## Learnable Rounding

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 8             # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 185.95
Epoch 1, Validation Loss: 163.09
Epoch 2, Validation Loss: 127.20
Epoch 3, Validation Loss: 12.72
Epoch 4, Validation Loss: 16.47
Epoch 5, Validation Loss: 13.96
Epoch 6, Validation Loss: 14.41
Epoch 7, Validation Loss: 14.16
Epoch 8, Validation Loss: 12.18
Epoch 9, Validation Loss: 12.99
Epoch 10, Validation Loss: 13.55
Epoch 11, Validation Loss: 9.56
Epoch 12, Validation Loss: 11.76
Epoch 13, Validation Loss: 11.88
Epoch 14, Validation Loss: 10.24
Epoch 15, Validation Loss: 9.83
Epoch 16, Validation Loss: 10.01
Epoch 17, Validation Loss: 12.34
Epoch 18, Validation Loss: 12.48
Epoch 19, Validation Loss: 14.20
Epoch 20, Validation Loss: 10.19
Epoch 21, Validation Loss: 14.13
Epoch 22, Validation Loss: 10.78
Epoch 23, Validation Loss: 9.79
Epoch 24, Validation Loss: 10.28
Epoch 25, Validation Loss: 10.31
Epoch 26, Validation Loss: 9.94
Epoch 27, Validation Loss: 10.33
Epoch 28, Validation Loss: 9.23
Epoch 29, Validation Loss: 8.37
Epoch 30, Validation Lo

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-1.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 314.50it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     9.685436          0.010450      0.002480
std      9.993887          0.081436      0.000576
min      0.110410          0.000000      0.001504
25%      2.526931          0.000000      0.002000
50%      6.413675          0.000000      0.002504
75%     12.941375          0.000000      0.003000
max     48.295884          0.808981      0.004495
Number of infeasible solution: 7


## Learnable Threshold

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 8             # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 185.95
Epoch 1, Validation Loss: 174.13
Epoch 2, Validation Loss: 113.96
Epoch 3, Validation Loss: 19.13
Epoch 4, Validation Loss: 9.99
Epoch 5, Validation Loss: 7.48
Epoch 6, Validation Loss: 7.32
Epoch 7, Validation Loss: 7.49
Epoch 8, Validation Loss: 7.06
Epoch 9, Validation Loss: 8.06
Epoch 10, Validation Loss: 6.86
Epoch 11, Validation Loss: 7.25
Epoch 12, Validation Loss: 6.70
Epoch 13, Validation Loss: 6.87
Epoch 14, Validation Loss: 6.79
Epoch 15, Validation Loss: 6.30
Epoch 16, Validation Loss: 6.47
Epoch 17, Validation Loss: 6.30
Epoch 18, Validation Loss: 6.17
Epoch 19, Validation Loss: 6.08
Epoch 20, Validation Loss: 6.10
Epoch 21, Validation Loss: 6.38
Epoch 22, Validation Loss: 6.55
Epoch 23, Validation Loss: 6.26
Epoch 24, Validation Loss: 5.98
Epoch 25, Validation Loss: 5.66
Epoch 26, Validation Loss: 6.26
Epoch 27, Validation Loss: 5.58
Epoch 28, Validation Loss: 5.57
Epoch 29, Validation Loss: 5.46
Epoch 30, Validation Loss: 5.38
Epoch 31, V

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-1.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 261.73it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean     5.125322               0.0      0.002977
std      4.409636               0.0      0.000954
min      0.111074               0.0      0.001510
25%      1.684655               0.0      0.002006
50%      4.114165               0.0      0.002999
75%      7.702470               0.0      0.003513
max     26.133279               0.0      0.005774
Number of infeasible solution: 0


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 80   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 8             # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x"], steepness, num_blocks, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 167.33
Epoch 1, Validation Loss: 154.45
Epoch 2, Validation Loss: 109.40
Epoch 3, Validation Loss: 5.06
Epoch 4, Validation Loss: 3.37
Epoch 5, Validation Loss: 2.65
Epoch 6, Validation Loss: 2.46
Epoch 7, Validation Loss: 2.47
Epoch 8, Validation Loss: 2.39
Epoch 9, Validation Loss: 2.33
Epoch 10, Validation Loss: 2.30
Epoch 11, Validation Loss: 2.26
Epoch 12, Validation Loss: 2.22
Epoch 13, Validation Loss: 2.15
Epoch 14, Validation Loss: 2.11
Epoch 15, Validation Loss: 2.05
Epoch 16, Validation Loss: 1.99
Epoch 17, Validation Loss: 1.97
Epoch 18, Validation Loss: 1.92
Epoch 19, Validation Loss: 1.89
Epoch 20, Validation Loss: 1.91
Epoch 21, Validation Loss: 1.86
Epoch 22, Validation Loss: 1.85
Epoch 23, Validation Loss: 1.86
Epoch 24, Validation Loss: 1.84
Epoch 25, Validation Loss: 1.82
Epoch 26, Validation Loss: 1.85
Epoch 27, Validation Loss: 1.82
Epoch 28, Validation Loss: 1.83
Epoch 29, Validation Loss: 1.82
Epoch 30, Validation Loss: 1.82
Epoch 31, Va

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x"]
    for i in range(num_blocks*2):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_pr_50-1.csv")
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 570.82it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     6.483303          0.000071      0.000914
std      3.959775          0.000553      0.000620
min      0.150241          0.000000      0.000000
25%      2.890288          0.000000      0.000510
50%      6.299332          0.000000      0.001000
75%      9.587027          0.000000      0.001136
max     16.273808          0.005204      0.002519
Number of infeasible solution: 2
